In [ ]:
! pip install datasets
! pip install transformers
! pip install accelerate -U

In [ ]:
pip install xformers

In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
model_path= 'distilbert-base-uncased'
tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path, id2label={0:"NEG", 1:"POS"},
label2id={"NEG":0, "POS":1})

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from datasets import load_dataset
imdb_train= load_dataset('imdb', split="train")
imdb_test= load_dataset('imdb', split="test[:6250]+test[-6250:]")
imdb_val= load_dataset('imdb', split="test[6250:12500]+test[-12500:-6250]")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
imdb_train.shape, imdb_test.shape, imdb_val.shape

((25000, 2), (12500, 2), (12500, 2))

In [6]:
imdb_train= load_dataset('imdb', split="train[:2000]+train[-2000:]")
imdb_test= load_dataset('imdb',split="test[:500]+test[-500:]")
imdb_val= load_dataset('imdb', split="test[500:1000]+test[-1000:-500]")

In [7]:
train = imdb_train.map(lambda e: tokenizer(e['text'], padding=True, truncation=True), batched=True, batch_size=1000)
test = imdb_test.map(lambda e: tokenizer( e['text'],padding=True, truncation=True), batched=True, batch_size=1000)
val = imdb_val.map(lambda e: tokenizer( e['text'],padding=True, truncation=True), batched=True, batch_size=1000)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
import pandas as pd
pd.DataFrame(train)

,text,label,input_ids,attention_mask
0,I rented I AM CURIOUS-YELLOW from my video sto...,0,"[101, 1045, 12524, 1045, 2572, 8025, 1011, 375...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"""I Am Curious: Yellow"" is a risible and preten...",0,"[101, 1000, 1045, 2572, 8025, 1024, 3756, 1000...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,If only to avoid making this type of film in t...,0,"[101, 2065, 2069, 2000, 4468, 2437, 2023, 2828...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,This film was probably inspired by Godard's Ma...,0,"[101, 2023, 2143, 2001, 2763, 4427, 2011, 2643...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"Oh, brother...after hearing about this ridicul...",0,"[101, 2821, 1010, 2567, 1012, 1012, 1012, 2044...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...
3995,A hit at the time but now better categorised a...,1,"[101, 1037, 2718, 2012, 1996, 2051, 2021, 2085...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3996,I love this movie like no other. Another time ...,1,"[101, 1045, 2293, 2023, 3185, 2066, 2053, 2060...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3997,This film and it's sequel Barry Mckenzie holds...,1,"[101, 2023, 2143, 1998, 2009, 1005, 1055, 8297...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3998,'The Adventures Of Barry McKenzie' started lif...,1,"[101, 1005, 1996, 7357, 1997, 6287, 18506, 100...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [10]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./MyIMDBModel',
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy='steps',
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy= 'steps',
    fp16= cuda.is_available(),
    load_best_model_at_end=True)

In [12]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  Precision, Recall, f1, _ =   precision_recall_fscore_support(labels, preds, average='macro')
  acc = accuracy_score(labels, preds)
  return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': Precision,
        'Recall': Recall
    }

In [13]:
trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train,
            eval_dataset=val,
            compute_metrics= compute_metrics
            )

In [14]:
results=trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,0.480100,0.419500,0.812000,0.807187,0.846611,0.812000
200,0.246600,0.380198,0.870000,0.869284,0.878286,0.870000
300,0.152400,0.379983,0.892000,0.891791,0.895059,0.892000


In [15]:
q=[trainer.evaluate(eval_dataset=data) for data in[train, val, test]]

pd.DataFrame(q, index=["train","val","test"]).iloc[:,:5]

,eval_loss,eval_Accuracy,eval_F1,eval_Precision,eval_Recall
train,0.048438,0.9895,0.9895,0.989502,0.9895
val,0.345818,0.8940,0.8940,0.894006,0.8940
test,0.279253,0.9080,0.9080,0.908000,0.9080


In [16]:
def get_prediction(text):
    inputs = tokenizer(text, padding=True,truncation=True, max_length=250, return_tensors="pt").to(device)
    outputs = model(inputs["input_ids"].to(device),inputs["attention_mask"].to(device))
    probs = outputs[0].softmax(1)
    return probs, probs.argmax()

In [17]:
text = "I didn't like the movie it bored me "
get_prediction(text)[1].item()

0

In [18]:
model_save_path = "MyBestIMDBModel"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

('MyBestIMDBModel/tokenizer_config.json',
 'MyBestIMDBModel/special_tokens_map.json',
 'MyBestIMDBModel/vocab.txt',
 'MyBestIMDBModel/added_tokens.json',
 'MyBestIMDBModel/tokenizer.json')

In [19]:
from transformers import pipeline , DistilBertForSequenceClassification, DistilBertTokenizerFast

model = DistilBertForSequenceClassification.from_pretrained("MyBestIMDBModel")

tokenizer= DistilBertTokenizerFast.from_pretrained("MyBestIMDBModel")
SA_model= pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
SA_model("the movie was very impressive")


[{'label': 'POS', 'score': 0.988036036491394}]